In [15]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Input
from tensorflow.keras.optimizers import Adam
import json

# Step 1: Load the dataset
print("Loading dataset...")
file_path = r'C:\Users\as998\OneDrive\Documents\Submission4_Group13\Dataset\opcodes_dataset.csv'
opcodes_data = pd.read_csv(file_path)
print(f"Dataset loaded: {len(opcodes_data)} samples")

# Step 2: Tokenize Opcodes
print("Tokenizing opcodes...")
opcodes = opcodes_data['opcodes'].str.split(',').explode().unique()
opcode_to_index = {opcode: idx + 1 for idx, opcode in enumerate(opcodes)}  # Reserve 0 for padding
opcodes_data['tokenized_opcodes'] = opcodes_data['opcodes'].apply(
    lambda x: [opcode_to_index.get(opcode, 0) for opcode in x.split(',')]
)
print(f"Tokenization completed! Unique opcodes: {len(opcode_to_index)}")

# Step 3: Pad/Truncate sequences
print("Padding sequences...")
MAX_SEQ_LENGTH = 5000

def pad_sequences_manual(sequences, maxlen, padding='post', truncating='post', value=0):
    padded_sequences = []
    for seq in sequences:
        if len(seq) > maxlen:
            seq = seq[:maxlen] if truncating == 'post' else seq[-maxlen:]
        elif len(seq) < maxlen:
            seq = seq + [value] * (maxlen - len(seq)) if padding == 'post' else [value] * (maxlen - len(seq)) + seq
        padded_sequences.append(seq)
    return np.array(padded_sequences)

padded_opcodes = pad_sequences_manual(
    opcodes_data['tokenized_opcodes'], maxlen=MAX_SEQ_LENGTH, padding='post', truncating='post', value=0
)
print(f"Padding completed! Shape of padded sequences: {padded_opcodes.shape}")

# Step 4: Encode Labels
print("Encoding labels...")
opcodes_data['apt'] = opcodes_data['apt'].astype(int)

# Step 5: Merge Rare Classes into 'Other'
print("Merging rare classes into 'Other'...")
class_counts = opcodes_data['apt'].value_counts()
rare_classes = class_counts[class_counts < 5].index
opcodes_data['apt'] = opcodes_data['apt'].apply(lambda x: -1 if x in rare_classes else x)
unique_classes = np.unique(opcodes_data['apt'])
class_mapping = {cls: idx for idx, cls in enumerate(unique_classes)}
opcodes_data['apt'] = opcodes_data['apt'].map(class_mapping)
print(f"Class mapping: {class_mapping}")

# Step 6: Prepare the dataset
print("Preparing the dataset...")
X = padded_opcodes
y = opcodes_data['apt'].values

# Step 7: Perform Stratified Split (Fallback to Random Split if Stratified Fails)
try:
    print("Performing stratified split...")
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
    )
except ValueError as e:
    print(f"Stratified split failed: {e}")
    print("Falling back to random split...")
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.3, random_state=42
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42
    )

print("Saving dataset splits...")
np.save('X_train.npy', X_train)
np.save('y_train.npy', y_train)
np.save('X_val.npy', X_val)
np.save('y_val.npy', y_val)
np.save('X_test.npy', X_test)
np.save('y_test.npy', y_test)

# Save opcode-to-index mapping
print("Saving opcode-to-index mapping...")
with open('opcode_to_index.json', 'w') as f:
    json.dump(opcode_to_index, f)
print("Opcode-to-index mapping saved.")

# Save class mapping
print("Saving class mapping...")
with open('class_mapping.json', 'w') as f:
    json.dump({int(k): v for k, v in class_mapping.items()}, f)
print("Class mapping saved.")

# Dataset Summary
print("\nDataset Summary:")
print(f"Training Set Size: {len(X_train)}")
print(f"Validation Set Size: {len(X_val)}")
print(f"Test Set Size: {len(X_test)}")
print(f"Number of Unique Opcodes: {len(opcode_to_index)}")
print(f"Sequence Length: {MAX_SEQ_LENGTH}")
print(f"Number of Unique Classes: {len(class_mapping)}")

# Step 8: Define the Model
print("\nDefining the model...")
vocab_size = len(opcode_to_index) + 1  # Vocabulary size including padding
embedding_dim = 8
sequence_length = MAX_SEQ_LENGTH
num_classes = len(class_mapping)  # Number of unique classes

model = Sequential([
    Input(shape=(sequence_length,)),
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    Conv1D(filters=64, kernel_size=8, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(16, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

# Step 9: Train the Model
print("\nTraining the model...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,  # Adjust epochs as needed
    batch_size=32,
    verbose=1
)

# Step 10: Save the Trained Model
# Save the trained model using the Keras native format
model.save('deep_malware_detection_model.keras')
print("Model saved as deep_malware_detection_model.keras") 

Loading dataset...
Dataset loaded: 90 samples
Tokenizing opcodes...
Tokenization completed! Unique opcodes: 448
Padding sequences...
Padding completed! Shape of padded sequences: (90, 5000)
Encoding labels...
Merging rare classes into 'Other'...
Class mapping: {np.int64(-1): 0, np.int64(7): 1, np.int64(10): 2, np.int64(11): 3, np.int64(16): 4, np.int64(19): 5, np.int64(32): 6}
Preparing the dataset...
Performing stratified split...
Stratified split failed: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.
Falling back to random split...
Saving dataset splits...
Saving opcode-to-index mapping...
Opcode-to-index mapping saved.
Saving class mapping...
Class mapping saved.

Dataset Summary:
Training Set Size: 63
Validation Set Size: 13
Test Set Size: 14
Number of Unique Opcodes: 448
Sequence Length: 5000
Number of Unique Classes: 7

Defining the model...


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)              │ (None, 5000, 8)             │           3,592 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_8 (Conv1D)                    │ (None, 4993, 64)            │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_8               │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 16)                  │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 7)                   │             119 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,911 (34.81 KB)

 Trainable params: 8,911 (34.81 KB)

 Non-trainable params: 0 (0.00 B)


Training the model...
Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 569ms/step - accuracy: 0.5056 - loss: 1.9392 - val_accuracy: 0.6154 - val_loss: 1.9303
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 0.6210 - loss: 1.9286 - val_accuracy: 0.6154 - val_loss: 1.9203
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step - accuracy: 0.6106 - loss: 1.9180 - val_accuracy: 0.6154 - val_loss: 1.9082
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - accuracy: 0.6106 - loss: 1.9058 - val_accuracy: 0.6154 - val_loss: 1.8935
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step - accuracy: 0.5481 - loss: 1.8947 - val_accuracy: 0.6154 - val_loss: 1.8766
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - accuracy: 0.6419 - loss: 1.8688 - val_accuracy: 0.6154 - val_loss: 1.8570
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step - accuracy: 0.5898 - loss: 1.8558 - val_accuracy: 0.6154 - val_loss: 1.8353
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step - accuracy: 0.6314 - loss: 1.8263 - val_accur